<a href="https://colab.research.google.com/github/fhswf/ki-wir/blob/main/VAEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational Autoencoder


In [1]:
# Setup on Colab
!pip install gradio &> /dev/null
!if [ ! -e ki_wir ]; then git clone https://github.com/fhswf/ki-wir.git ki_wir; else cd ki_wir; git pull; fi

Cloning into 'ki_wir'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 24 (delta 9), reused 10 (delta 3), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [2]:
import gradio as gr
import torch
import numpy as np
import ki_wir.models.simple_vae as vae

from PIL import Image
from torchvision import transforms

In [3]:
device = torch.device("cuda:0")
model = vae.VAE(is_train=False).to(device)
model.load_state_dict(torch.load("ki_wir/pretrained/vae-123.pth", map_location=device))

def reconstruct(original):
  img = Image.fromarray(original)
  img = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        ])(img)

  orig = transforms.ToPILImage(mode='RGB')(img)
  img = torch.unsqueeze(img.cuda(), 0)
  enc = model.encode(img)
  dec = model.decode(enc)
  dec = torch.squeeze(dec, 0)

  return [orig, transforms.ToPILImage(mode='RGB')(dec)]

In [8]:
out1 = gr.outputs.Image(type="auto", label="original")
out2 = gr.outputs.Image(type="auto", label="reconstructed")
iface = gr.Interface(fn=reconstruct, inputs="image", outputs=[out1, out2]).launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/utils.py:30: UserWarning: unable to connect with package URL to collect version info.
  warnings.warn("unable to connect with package URL to collect version info.")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().


KeyboardInterrupt: ignored